In [19]:
import sys
from datetime import datetime
from os.path import join
from warnings import warn


import numpy as np
import pandas as pd
import scipy as sp

from utils import *

In [20]:
# sys.argv[1] = 'test'

In [21]:
dir_arg = sys.argv[1]
if dir_arg == '-f':
    file_dir = join('..', 'dataset', '11')
else:
    file_dir = join('..', 'dataset',  dir_arg)

In [22]:
train_df = pd.read_pickle(join(file_dir, 'base_feauture.pkl'))

sample = pd.read_pickle(join(file_dir, 'hotelid.pkl'))

now_date = train_df.orderdate.max().date()
print(datetime.now(), now_date)

uid_shape, hotelid_shape, basicroomid_shape, roomid_shape = print_shape(
    train_df, ['uid', 'hotelid', 'basicroomid', 'roomid'])

2017-07-30 23:19:10.574333 2017-06-11
2017-07-30 23:19:10.574548
--------------------
uid uniuqe shape 3552
hotelid uniuqe shape 3194
basicroomid uniuqe shape 19542
roomid uniuqe shape 121802
--------------------


In [23]:
feature_path = join(file_dir, 'hotel_feature.pkl')
print(datetime.now(), 'begin', feature_path)

2017-07-30 23:19:14.158405 begin ../dataset/11/hotel_feature.pkl


## 添加基本特征

In [24]:
sample = add_column(train_df, sample, 'hotelid', 'star')

## 上下级关联统计特征

In [25]:
for f in ['basicroomid', 'roomid']:
    print(datetime.now(), 'begin hotel', f, 'count')
    sample = extract_feature_count('hotelid', f, train_df, sample)

2017-07-30 23:19:16.495146 begin hotel basicroomid count
2017-07-30 23:19:16.518571 begin hotel roomid count


In [26]:
for i in range(8):
    f = 'roomservice_%d' % (i+1)
    sample = extract_feature_count('hotelid', f, train_df, sample)

In [27]:
for i in range(1, 4):
    f = 'roomtag_%d' % (i+1)
    sample = extract_feature_count('hotelid', f, train_df, sample)

In [28]:
# get_corr(train_df, sample, 'hotelid')

### 删除无历史记录却有历史返现值的记录值(默认值为200）

In [29]:
lastord_cols = [x for x in train_df.columns if x.endswith('lastord')]

In [30]:
train_df.loc[train_df.orderdate_lastord.isnull(), 'return_lastord'] = np.nan

In [31]:
train_df.price_deduct.describe()

count    130801.000000
mean       2909.073242
std        8405.767068
min         236.000000
25%         986.000000
50%        1625.000000
75%        3050.000000
max      780200.000000
Name: price_deduct, dtype: float64

## 基本特征 

## 显示的最终价格和原价格的特征 

In [32]:
use_describe = ['max', 'min', '75%', 'mean', 'std']

In [33]:
train_df['price_real'] = train_df['price_deduct'] + train_df['returnvalue']

In [15]:
sample = extract_value_describe_feature('hotelid', 'price_deduct', train_df, sample, use_describe)

sample = extract_value_describe_feature('hotelid', 'price_real', train_df, sample, ['max', 'mean'])

sample = extract_value_describe_feature('hotelid', 'returnvalue', train_df, sample, ['max'])

### 房间的面积统计特征

###  删掉为负的值

In [5]:
train_df.loc[train_df.basic_minarea<0, 'basic_minarea'] = np.nan

In [6]:
sample = extract_value_describe_feature('hotelid', 'basic_minarea', train_df, sample, ['max', 'mean', '75%'])

sample = extract_value_describe_feature('hotelid', 'basic_maxarea', train_df, sample, ['min', 'mean', '25%'])

In [7]:
# get_corr(train_df, sample, 'hotelid').tail(25)

,orderlabel,hotelid__basic_minarea_count,hotelid__basic_minarea_mean,hotelid__basic_minarea_std,hotelid__basic_minarea_min,hotelid__basic_minarea_25,hotelid__basic_minarea_50,hotelid__basic_minarea_75,hotelid__basic_minarea_max,hotelid__basic_maxarea_count,hotelid__basic_maxarea_mean,hotelid__basic_maxarea_std,hotelid__basic_maxarea_min,hotelid__basic_maxarea_25,hotelid__basic_maxarea_50,hotelid__basic_maxarea_75,hotelid__basic_maxarea_max
orderlabel,1.000000,-0.067539,-0.037418,-0.022417,-0.031092,-0.035304,-0.035178,-0.035744,-0.024206,-0.067572,-0.037986,-0.022187,-0.029952,-0.035827,-0.035924,-0.036576,-0.024215
hotelid__basic_minarea_count,-0.067539,1.000000,0.245353,0.127404,0.257888,0.268272,0.226846,0.237753,0.156665,0.999891,0.252323,0.124917,0.264485,0.271103,0.229273,0.251200,0.155020
hotelid__basic_minarea_mean,-0.037418,0.245353,1.000000,0.772048,0.765017,0.805037,0.830183,0.887485,0.756554,0.244511,0.992486,0.768745,0.731682,0.789708,0.819059,0.891933,0.756177
hotelid__basic_minarea_std,-0.022417,0.127404,0.772048,1.000000,0.271730,0.301358,0.348187,0.543221,0.968218,0.127047,0.770233,0.996036,0.263592,0.298408,0.347233,0.569096,0.967456
hotelid__basic_minarea_min,-0.031092,0.257888,0.765017,0.271730,1.000000,0.944787,0.860728,0.714538,0.323481,0.256937,0.755348,0.270279,0.950421,0.923383,0.846329,0.702543,0.323360
hotelid__basic_minarea_25,-0.035304,0.268272,0.805037,0.301358,0.944787,1.000000,0.911481,0.769510,0.350814,0.267289,0.795811,0.299627,0.900732,0.975599,0.896623,0.761039,0.350551
hotelid__basic_minarea_50,-0.035178,0.226846,0.830183,0.348187,0.860728,0.911481,1.000000,0.818950,0.364135,0.225941,0.820163,0.345290,0.820120,0.893735,0.978715,0.807065,0.363664
hotelid__basic_minarea_75,-0.035744,0.237753,0.887485,0.543221,0.714538,0.769510,0.818950,1.000000,0.520857,0.236951,0.879749,0.539622,0.684276,0.755250,0.810810,0.987744,0.520275
hotelid__basic_minarea_max,-0.024206,0.156665,0.756554,0.968218,0.323481,0.350814,0.364135,0.520857,1.000000,0.156361,0.754601,0.963875,0.311893,0.346322,0.362190,0.549567,0.998785
hotelid__basic_maxarea_count,-0.067572,0.999891,0.244511,0.127047,0.256937,0.267289,0.225941,0.236951,0.156361,1.000000,0.250778,0.124772,0.260933,0.268728,0.227586,0.249995,0.154642


## 过去物理房型和子房型的统计特征 

In [ ]:
# get_corr(train_df, sample, 'hotelid')

In [8]:
basic_cols = [
    'basic_week_ordernum_ratio', 'basic_recent3_ordernum_ratio',
    'basic_comment_ratio', 'basic_30days_ordnumratio', 'basic_30days_realratio'
]

In [ ]:
stat_describe = ['min', 'mean']

In [9]:
for col in basic_cols:
    sample = extract_value_describe_feature('hotelid', col, train_df, sample, stat_describe)

/home/zhanglun/notebook/room/code/utils.py:30: UserWarning: column  hotelid__basic_week_ordernum_ratio_std is  may be error when meet percent max:0.7071067811865476
  warn('column  {} is  may be error when meet percent max:{}'.format(c, c_max))
/home/zhanglun/notebook/room/code/utils.py:30: UserWarning: column  hotelid__basic_recent3_ordernum_ratio_std is  may be error when meet percent max:0.7071067811865476
  warn('column  {} is  may be error when meet percent max:{}'.format(c, c_max))
/home/zhanglun/notebook/room/code/utils.py:30: UserWarning: column  hotelid__basic_30days_ordnumratio_std is  may be error when meet percent max:0.4963735677390567
  warn('column  {} is  may be error when meet percent max:{}'.format(c, c_max))


In [11]:
room_cols = ['room_30days_ordnumratio', 'room_30days_realratio']

In [12]:
for col in room_cols:
    sample = extract_value_describe_feature('hotelid', col, train_df, sample, ['max', 'min'])

/home/zhanglun/notebook/room/code/utils.py:30: UserWarning: column  hotelid__room_30days_ordnumratio_std is  may be error when meet percent max:0.606202505404434
  warn('column  {} is  may be error when meet percent max:{}'.format(c, c_max))


In [15]:
name_fmt = '{}_diff_{}'.format('hotelid', '{}')

price_diff_name = name_fmt.format('price_last_lastord')
hotel_minprice_diff_name = name_fmt.format('hotel_minprice_lastord')
basic_minprice_diff_name = name_fmt.format('basic_minprice_lastord')

In [16]:
train_df[price_diff_name] = train_df['price_deduct'] - train_df['price_last_lastord']
train_df[hotel_minprice_diff_name] = train_df['price_deduct'] - train_df['hotel_minprice_lastord']
train_df[basic_minprice_diff_name] = train_df['price_deduct'] - train_df['basic_minprice_lastord']

In [ ]:
price_desr = ['mean', 'max']

In [17]:
sample = extract_value_describe_feature('hotelid', price_diff_name, train_df, sample, price_desr)
sample = extract_value_describe_feature('hotelid', hotel_minprice_diff_name, train_df, sample, price_desr)
sample = extract_value_describe_feature('hotelid', basic_minprice_diff_name, train_df, sample, price_desr)

In [18]:
get_corr(train_df, sample, 'hotelid').tail(24)

,orderlabel,hotelid__basic_minarea_count,hotelid__basic_minarea_mean,hotelid__basic_minarea_std,hotelid__basic_minarea_min,hotelid__basic_minarea_25,hotelid__basic_minarea_50,hotelid__basic_minarea_75,hotelid__basic_minarea_max,hotelid__basic_maxarea_count,...,hotelid__hotelid_diff_hotel_minprice_lastord_75,hotelid__hotelid_diff_hotel_minprice_lastord_max,hotelid__hotelid_diff_basic_minprice_lastord_count,hotelid__hotelid_diff_basic_minprice_lastord_mean,hotelid__hotelid_diff_basic_minprice_lastord_std,hotelid__hotelid_diff_basic_minprice_lastord_min,hotelid__hotelid_diff_basic_minprice_lastord_25,hotelid__hotelid_diff_basic_minprice_lastord_50,hotelid__hotelid_diff_basic_minprice_lastord_75,hotelid__hotelid_diff_basic_minprice_lastord_max
hotelid__hotelid_diff_price_last_lastord_count,-0.066666,0.962152,0.236222,0.123284,0.253809,0.258298,0.216258,0.231311,0.150631,0.962312,...,0.453677,0.190634,1.000000,0.311055,0.166564,0.037686,0.263971,0.389445,0.431263,0.190778
hotelid__hotelid_diff_price_last_lastord_mean,-0.027135,0.306436,0.465838,0.501409,0.273207,0.268959,0.260917,0.367365,0.482860,0.306320,...,0.808194,0.796307,0.307817,0.999291,0.817156,0.561359,0.737695,0.824460,0.863647,0.804183
hotelid__hotelid_diff_price_last_lastord_std,-0.019606,0.177602,0.505282,0.657719,0.231495,0.221652,0.223906,0.329551,0.649335,0.177395,...,0.519361,0.977584,0.166911,0.819949,0.999995,0.118549,0.269018,0.387802,0.481200,0.976996
hotelid__hotelid_diff_price_last_lastord_min,0.000630,-0.006525,0.022163,0.028805,0.022329,0.022029,0.009476,0.024652,0.039237,-0.006324,...,0.395914,0.137924,0.023415,0.540410,0.104465,0.995773,0.922017,0.754253,0.611171,0.155981
hotelid__hotelid_diff_price_last_lastord_25,-0.016081,0.230850,0.154962,0.114047,0.140605,0.140253,0.119683,0.155936,0.124013,0.231016,...,0.648224,0.286416,0.252154,0.723918,0.257778,0.934246,0.996835,0.920921,0.823266,0.302598
hotelid__hotelid_diff_price_last_lastord_50,-0.027036,0.380456,0.266937,0.199347,0.228888,0.226455,0.199609,0.255611,0.204385,0.380477,...,0.810012,0.398535,0.384456,0.817756,0.379984,0.772534,0.929832,0.998151,0.943078,0.412260
hotelid__hotelid_diff_price_last_lastord_75,-0.034016,0.426404,0.392940,0.305907,0.301657,0.303681,0.281791,0.397230,0.298344,0.426268,...,0.929641,0.482051,0.429742,0.860910,0.475895,0.629517,0.835215,0.947510,0.998953,0.491495
hotelid__hotelid_diff_price_last_lastord_max,-0.021158,0.202219,0.497534,0.655924,0.234797,0.224220,0.216979,0.325275,0.676148,0.202048,...,0.521337,0.999607,0.190593,0.806945,0.976991,0.170697,0.314296,0.419837,0.496674,0.999994
hotelid__hotelid_diff_hotel_minprice_lastord_count,-0.066666,0.962152,0.236222,0.123284,0.253809,0.258298,0.216258,0.231311,0.150631,0.962312,...,0.453677,0.190634,1.000000,0.311055,0.166564,0.037686,0.263971,0.389445,0.431263,0.190778
hotelid__hotelid_diff_hotel_minprice_lastord_mean,-0.029682,0.319781,0.509334,0.529324,0.311393,0.304792,0.299149,0.414117,0.508111,0.319623,...,0.851738,0.834980,0.321602,0.955515,0.860636,0.370986,0.581866,0.708102,0.797908,0.834776


## 历史价格与现在差价统计特征

In [24]:
hotel_lastord = train_df[[
    'uid', 'hotelid_lastord', 'hotelid', 'star_lastord',
    'hotel_minprice_lastord', 'orderdate_lastord'
]].drop_duplicates()

In [25]:
if uid_shape != hotel_lastord.shape[0]:
    warn('uid_shape not equal [uid ,hotelid_lastord, hotelid]')

## 历史购买时间间隔统计特征 

In [64]:
span_name, t = '{}_span'.format('hotelid'), 'hotelid'

In [65]:
# train_df[span_name] = (now_date - train_df.orderdate_lastord).dt.days

# sample = extract_value_describe_feature(t, span_name, train_df, sample, ['max', 'min', 'mean'])

In [69]:
# get_corr(train_df, sample, 'hotelid').tail(8)

## 用户过去是否购买或者没有记录

In [26]:
sample = extract_lastord_is_nan(hotel_lastord, sample, 'hotelid', 'hotelid_lastord')

In [27]:
sample = extract_is_lastord(hotel_lastord, sample, 'hotelid', 'hotelid_lastord')

In [28]:
# get_corr(train_df, sample, 'hotelid').tail()

In [29]:
def extract_lastord_feature_max_min(t, hotel_lastord, sample):
    min_fmt = '{}_min'.format(t)
    max_fmt = '{}_max'.format(t)
    series = hotel_lastord.groupby('hotelid')[t].min()
    series.name = min_fmt
    sample = sample.join(series, on='hotelid')
    series = hotel_lastord.groupby('hotelid')[t].max()
    series.name = max_fmt
    sample = sample.join(series, on='hotelid')
    max_equal_min_fmt = '{}_max_equeal_min'.format(t)
    sample[max_equal_min_fmt] = (sample[max_fmt] == sample[min_fmt]).astype(np.int8)
    sample = press_date(sample, [max_equal_min_fmt, max_fmt, min_fmt])
    return sample

In [30]:
# sample = extract_lastord_feature_max_min('star_lastord', hotel_lastord, sample)

# sample = extract_lastord_feature_max_min('hotel_minprice_lastord', hotel_lastord, sample)

# hotel_lastord['orderdate_lastord_days'] = (now_date - hotel_lastord.orderdate_lastord).dt.days

# sample = extract_lastord_feature_max_min('orderdate_lastord_days', hotel_lastord, sample)

In [31]:
# get_corr(train_df, sample, 'hotelid')

In [32]:
# sample.info()

In [33]:
sample.to_pickle(feature_path)

print(datetime.now(), 'save to', feature_path)